In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import datasets
from torchinfo import summary
from pathlib import Path
from sklearn.model_selection import train_test_split
from scripts import data_acquisition, data_setup, engine, plot_loss_curves, utils, model_acquisition

In [ ]:
BATCH_SIZE = 256
EPOCHS = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
efficientnet_weights = models.EfficientNet_B2_Weights.DEFAULT
efficientnet_transforms = efficientnet_weights.transforms()
efficientnet_model = models.efficientnet_b2(weights=efficientnet_weights)

In [ ]:
resnet18_weights = models.ResNet18_Weights.DEFAULT
resnet18_transforms = resnet18_weights.transforms()
resnet18_model = models.resnet18(weights=resnet18_weights)

In [ ]:
resnet152_weights = models.ResNet152_Weights.DEFAULT
resnet152_transforms = resnet152_weights.transforms()
resnet152_model = models.resnet152(weights=resnet152_weights)

In [ ]:
vit_weights = models.ViT_B_16_Weights.DEFAULT
vit_transforms = vit_weights.transforms()
vitb16_model = models.vit_b_16(weights=vit_weights)

In [ ]:
# Run once to download the dataset
traindataset = datasets.Food101(root = 'data', download = True, transform = None, split = "train")
testdataset = datasets.Food101(root = 'data', download = True, transform = None, split = "test")
print(len(traindataset), len(testdataset))

### Freeze Parameters

In [ ]:
for param in efficientnet_model.parameters():
    param.requires_grad = False
# print(summary(efficientnet_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params", "trainable")))
efficientnet_model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1408, out_features=101, bias=True)
)
# print(summary(efficientnet_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params", "trainable")))

In [ ]:
for param in resnet18_model.parameters():
    param.requires_grad = False

# print(summary(resnet18_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params")))
resnet18_model.fc = nn.Linear(in_features=512, out_features=101, bias=True)
# print(summary(resnet18_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params", "trainable")))

In [ ]:
for param in resnet152_model.parameters():
    param.requires_grad = False

# print(summary(resnet152_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params")))
resnet152_model.fc =  nn.Linear(in_features=2048, out_features=101, bias=True)
# print(summary(resnet152_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params", "trainable")))

In [ ]:
for param in vitb16_model.parameters():
    param.requires_grad = False
    
# print(summary(vitb16_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params")))
vitb16_model.heads.head = nn.Linear(in_features=768, out_features=101, bias=True)
# print(summary(vitb16_model, input_size=(32, 3, 224, 224),col_names=("input_size", "output_size", "num_params", "trainable")))

### Train Efficientnet

In [ ]:
traindataset = datasets.CIFAR10(root = 'data', download = True, split = "train", transform = efficientnet_transforms)
testdataset = datasets.CIFAR10(root = 'data', download = True,  split = "test", transform = efficientnet_transforms)
print(efficientnet_transforms)

In [ ]:
traindataloader = DataLoader(dataset = traindataset,
                            batch_size = BATCH_SIZE,
                            shuffle = True,)

testdataloader = DataLoader(dataset = testdataset,
                            batch_size = BATCH_SIZE,
                            shuffle = False)

In [ ]:
history, all_preds, all_labels, writer = engine.train(model = efficientnet_model.to(device),
                                                    model_name = "efficientnet",
                                                    data_name="Food101",
                                                    epochs = EPOCHS,
                                                    train_dataloader=traindataloader,
                                                    val_dataloader = testdataloader,
                                                    device = device,
                                                    loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1),
                                                    optimizer = torch.optim.Adam(efficientnet_model.parameters())
                                                    )

In [ ]:
torch.save(efficientnet_model.state_dict(), 'efficientnet_model_Food101.pth')

### Train Resnet 18

In [ ]:
traindataset = datasets.CIFAR10(root = 'data', download = True, split = "train", transform = resnet18_transforms)
testdataset = datasets.CIFAR10(root = 'data', download = True, split = "test", transform = resnet18_transforms)
print(efficientnet_transforms)

In [ ]:
traindataloader = DataLoader(dataset = traindataset,
                            batch_size = BATCH_SIZE,
                            shuffle = True,)

testdataloader = DataLoader(dataset = testdataset,
                            batch_size = BATCH_SIZE,
                            shuffle = False)

In [ ]:
history, all_preds, all_labels, writer = engine.train(model = resnet18_model.to(device),
                                                    model_name = "resnet_18",
                                                    data_name="Food101",
                                                    epochs = EPOCHS,
                                                    train_dataloader=traindataloader,
                                                    val_dataloader = testdataloader,
                                                    device = device,
                                                    loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1),
                                                    optimizer = torch.optim.Adam(efficientnet_model.parameters())
                                                    )

In [ ]:
torch.save(efficientnet_model.state_dict(), 'resnet_18_model_Food101.pth')

### Train Resnet 152

In [ ]:
traindataset = datasets.CIFAR10(root = 'data', download = True, split = "train", transform = resnet18_transforms)
testdataset = datasets.CIFAR10(root = 'data', download = True, split = "test", transform = resnet18_transforms)
print(efficientnet_transforms)

In [ ]:
traindataloader = DataLoader(dataset = traindataset,
                            batch_size = BATCH_SIZE,
                            shuffle = True,)

testdataloader = DataLoader(dataset = testdataset,
                            batch_size = BATCH_SIZE,
                            shuffle = False)

In [ ]:
history, all_preds, all_labels, writer = engine.train(model = resnet18_model.to(device),
                                                    model_name = "resnet_18",
                                                    data_name="Food101",
                                                    epochs = EPOCHS,
                                                    train_dataloader=traindataloader,
                                                    val_dataloader = testdataloader,
                                                    device = device,
                                                    loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1),
                                                    optimizer = torch.optim.Adam(efficientnet_model.parameters())
                                                    )

### Train ViT B 16

In [ ]:
traindataset = datasets.CIFAR10(root = 'data', download = True, split = "train", transform = vit_transforms)
testdataset = datasets.CIFAR10(root = 'data', download = True, split = "test", transform = vit_transforms)
print(efficientnet_transforms)

In [ ]:
traindataloader = DataLoader(dataset = traindataset,
                            batch_size = BATCH_SIZE,
                            shuffle = True,)

testdataloader = DataLoader(dataset = testdataset,
                            batch_size = BATCH_SIZE,
                            shuffle = False)

In [ ]:
history, all_preds, all_labels, writer = engine.train(model = resnet152_model.to(device),
                                                    model_name = "vit_b_16",
                                                    data_name="Food101",
                                                    epochs = EPOCHS,
                                                    train_dataloader=traindataloader,
                                                    val_dataloader = testdataloader,
                                                    device = device,
                                                    loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1),
                                                    optimizer = torch.optim.Adam(efficientnet_model.parameters())
                                                    )

In [ ]:
torch.save(efficientnet_model.state_dict(), 'vit_b_16_model_Food101.pth')